In [ ]:
# MODULES IMPORTATION
import math
from math import *
import datetime
from datetime import *
import time
import sys
import os
from cmcrameri import cm
from pyproj import Geod

print("#################################")
print("# - Importing modules...")

try:
    print("# - Importing numpy")
    import numpy as np
    from numpy import *
except:
    raise ImportError(" ERROR importing numpy")
try:
    print("# - Importing pandas")
    import pandas as pd
except:
    raise ImportError(" ERROR importing pandas")
    
try:
    import geopandas as gpd
except:
    raise ImportError(" ERROR importing geopandas")
    
try:
    print("# - Importing pylab")
    import pylab
    from pylab import *
except:
    raise ImportError(" ERROR importing pylab")
try:
    print("# - Importing matplotlib")
    import matplotlib
    import matplotlib.pyplot as plt
    import matplotlib.colors as colors
    import matplotlib.gridspec as gridspec
    import matplotlib.lines as mlines
    from matplotlib.gridspec import GridSpec
    from mpl_toolkits.axes_grid1.anchored_artists import AnchoredSizeBar
    import matplotlib.font_manager as fm
    from cartopy import crs as ccrs, feature as cfeature
    from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER


except:
    raise ImportError(" ERROR importing matplotlib")
try:
    print("# - Importing scipy")
    import scipy
    from scipy import interpolate
    from scipy.interpolate import interp1d, interp2d
    from scipy.interpolate import griddata
    from scipy.stats import *
    import scipy.io as mio
    from scipy.spatial import distance
    from scipy.ndimage import gaussian_filter1d
    from scipy.optimize import minimize

except:
    raise ImportError(" ERROR importing scipy")
try:
    print("# - Importing gc")
    import gc
except:
    raise ImportError("  ERROR importing gc")
try:
    print("# - Importing socket")
    import socket
except:
    raise ImportError("  ERROR importing socket")
try:
    print("# - Importing getpass")
    import getpass
except:
    raise ImportError("  ERROR importing getpass")
try:
    import configparser
    from configparser import *
except:
    raise ImportError("  ERROR importing ConfigParser")
try:
    print("# - Importing statistics")
    import statistics
except:
    raise ImportError("  ERROR importing statistics")

try:
    print("# - Importing h5py")
    import h5py
except:
    raise ImportError("  ERROR importing h5py")
try:
    print("# -Importing xarray")
    import xarray as xr
except:
        raise ImportError("  ERROR importing xarray")
        
try:
    print("# - Importing cartopy")
    import cartopy
except:
    raise ImportError(" ERROR importing cartopy")

try:
    print("# - Importing globe")
    from global_land_mask import globe
except:
    raise ImportError(" ERROR importing globe")
    
try:
    print("# - Importing netcdf4")
    from netCDF4 import Dataset
except:
    raise ImportError(" ERROR importing netcdf")
    
from tqdm import tqdm


In [ ]:
# PATH DEFINITION TO TOOLBOXES DIRECTORY
try:
    print("# - Importing main toolboxes path...")
    global PTOOL
    # If path to TOOLBOXES is in you environment variables
    PTOOL = 'C:/Users/arias/ownCloud/GLOBCOAST/FUNCTION/'
except:
    raise ImportError(" ERROR path is not defined ")

# IMPORT OF TOOLBOXES
# (must respect dependancies between toolboxes)

print("# - Importing personal toolboxes...")
try:
    sys.path.append(PTOOL)
except:
    raise ImportError(" ERROR adding TOOLBOX PATH")
    
#  MAIN TOOLBOX
try:
    import function as FUNC    # toolbox where are all the functions I create for the model
except:
    raise ImportError(" ERROR importing FUNC")
    
try:
    import PYSTATS as PYSTATS    # toolbox where are all the functions I create for the model
except:
    raise ImportError(" ERROR importing FUNC")
    
print("# - Toolboxes are OK")

In [ ]:
def calculer_climatologie_saisonniere(dataframes, date_col='date'):
    climatologies_saisonnieres = {}
    
    for name, df in dataframes.items():
        # Assurez-vous que la colonne de date est en index si ce n'est pas déjà le cas
        if date_col in df.columns:
            df.set_index(date_col, inplace=True)
        
        # Extraire le mois et ajouter une colonne 'Mois'
        df['Mois'] = df.index.month
        
        # Calculer la moyenne par mois
        climatologies_saisonnieres[name] = df.groupby('Mois').mean()
    
    return climatologies_saisonnieres

def moving_average(data, window_size):
    return np.convolve(data, np.ones(window_size) / window_size, mode='same')

# Fonction pour calculer la distance entre deux points de latitude/longitude
def haversine(coord1, coord2):
    # Convertir les degrés en radians
    lat1, lon1 = radians(coord1[0]), radians(coord1[1])
    lat2, lon2 = radians(coord2[0]), radians(coord2[1])

    # Calcul de la différence entre les points
    dlat = lat2 - lat1
    dlon = lon2 - lon1

    # Formule de Haversine
    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))
    distance = 6371 * c  # Rayon de la Terre en kilomètres
    return distance

# Algorithme du plus proche voisin
def nearest_neighbor(coords):
    if not coords:
        return []

    path = [coords.pop(0)]  # Commence avec le premier point
    while coords:
        # Trouve le point le plus proche du dernier point ajouté au chemin
        next_point = min(coords, key=lambda x: haversine(path[-1], x))
        path.append(next_point)
        coords.remove(next_point)

    return path

def lonlat2xy(lon, lat, lon_0, lat_0):
    """
    lonlat2xy converts pairs of longitude and latitude into Cartesian
    coordinates x and y with lon_0 and lat_0 representing the coordinate
    system origin. 
    """
    geod = Geod(ellps="WGS84")
    lon_0_ar, lat_0_ar = np.full_like(lon, lon_0), np.full_like(lat, lat_0)

    # Calculate y coordinates 
    yd = np.where(lat < lat_0, -1, 1) * geod.inv(lon_0_ar, lat_0_ar, lon_0_ar, lat)[2]

    # Calculate x coordinates 
    xd = np.where(lon < lon_0, -1, 1) * geod.inv(lon_0_ar, lat_0_ar, lon, lat_0_ar)[2]

    return xd, yd

def xy2lonlat(x, y, lon_0, lat_0):
    """
    xy2lonlat converts pairs of Cartesian coordinates x and y back to
    longitude and latitude with lon_0 and lat_0 representing the coordinate
    system origin. Longitude and latitude are returned in degrees.
    """

    # Create Geod object for distance calculation
    geod = Geod(ellps="WGS84")

    lon_0_ar, lat_0_ar = np.full_like(x, lon_0), np.full_like(y, lat_0)

    # Calculate longitudes
    lon = geod.fwd(lon_0_ar, lat_0_ar, np.full_like(x, 90), x)[0]

    # Calculate latitudes
    lat = geod.fwd(lon_0_ar, lat_0_ar, np.zeros_like(y), y)[1]

    return lon, lat


def calculer_azimut(lat, lon):
    """
    Calcule l'azimut entre les points successifs spécifiés par leurs coordonnées de latitude et de longitude.

    Parameters:
    lat_ok (array-like): Tableau des latitudes des points.
    lon_ok (array-like): Tableau des longitudes des points.

    Returns:
    numpy.ndarray: Tableau des azimuts entre les points successifs.
    """
    alpha = np.zeros(len(lon) - 1)

    for i in range(len(lon) - 1):
        lat1 = np.radians(lat[i])
        lat2 = np.radians(lat[i+1])
        d_lon = np.radians(lon[i+1] - lon[i])

        x_azimuth = np.cos(lat1) * np.sin(lat2) - np.sin(lat1) * np.cos(lat2) * np.cos(d_lon)
        y_azimuth = np.sin(d_lon) * np.cos(lat2)

        alpha[i] = np.arctan2(y_azimuth, x_azimuth)

    return alpha

def calculer_normale_orientee(alpha, Dir):
    """
    Calcule les normales orientées en fonction de Dir.

    Parameters:
    alpha (numpy.ndarray): Tableau des azimuts.
    Dir (numpy.ndarray): Tableau des directions de vagues pour chaque point (ou zone).

    Returns:
    numpy.ndarray: Tableau des normales orientées.
    """
    # Convertir les angles en radians si nécessaire
    Dir_rad = np.radians(Dir)

    # Calcul des normales perpendiculaires
    normal_1 = (alpha + np.pi/2) % (2*np.pi)
    normal_2 = (alpha - np.pi/2) % (2*np.pi)
    normal_finale = np.zeros_like(alpha)

    for i in range(len(normal_1)):
        if abs(normal_1[i] - Dir_rad[i]) < abs(normal_2[i] - Dir_rad[i]):
            normal_finale[i] = normal_1[i]
        else:
            normal_finale[i] = normal_2[i]

    return normal_finale


In [ ]:
# FILES IMPORT

print("# - Retrieving Main files...")
pathin = 'C:/Users/arias/ownCloud/DATAS/Fichier_seul/' 
pathout = 'C:/Users/arias/ownCloud/GLOBCOAST/FIGURES/'
# ---------------------------------------------------------------------------------------
print("# - Uploading files ...")
# ---------------------------------------------------------------------------------------
# BQART DATA
Bqart_file = 'Sorties_Bqart.txt'
BQART_brut   = np.loadtxt(pathin+ Bqart_file, delimiter=';')
BQART_m3     = BQART_brut[:,0][:8841]/2650 #on divise par la densité du sable --> m3/an
BQART        = BQART_m3/12            # m3/yr --> m3/month
# ---------------------------------------------------------------------------------------
# SEADATAS FROM JULIEN
SEADATA = xr.open_dataset(pathin+'SEADATA_14140pts_1993_2019-analysed.nc',engine='netcdf4')
SEADATA_lon = SEADATA['lon'].values
SEADATA_lat = SEADATA['lat'].values
SEADATA_Hs  = SEADATA['Hs_mounthly'].values
SEADATA_Tp  = SEADATA['Tp_mounthly'].values
SEADATA_Dir = SEADATA['dir_mounthly'].values
SEADATA_SLA = SEADATA['sla_detrend'].values
SEADATA_DAC = SEADATA['dac_detrend'].values
SEADATA_RIV = SEADATA['rivdis_mounthly'].values
lon = SEADATA_lon[:8841]
lat = SEADATA_lat[:8841]
Hs  = SEADATA_Hs[84:,:8841]
Tp  = SEADATA_Tp[84:,:8841]
Dir = SEADATA_Dir[84:,:8841]
SLA = SEADATA_SLA[84:,:8841]
DAC = SEADATA_DAC[84:,:8841]
RiverD = SEADATA_RIV[84:,:8841]
# ---------------------------------------------------------------------------------------

TIDE = scipy.io.loadmat('C:/Users/arias/ownCloud/GLOBCOAST/DATAS/Tide_Glob.mat')
Marnage = TIDE['Tide_max'][0][:8841]

# ---------------------------------------------------------------------------------------
# # Validation file  (8841(position) x 444(time) --> 1984 to 2020 monthly) -> select 1993 to 2019
Validation     = scipy.io.loadmat(pathin+'Shorelines_global_20231101_shift.mat')
Xshores_val    = Validation['X_safe'][:,192:-12]
latX = Validation['latX'][:]
lonX = Validation['lonX'][:]

Xshores_VALIDATION = np.transpose(Xshores_val)
print("# - All files are upload...")


In [ ]:
print("# - Initializing constant values...")
# Median grain size [m]
d50 = 10e-3

# Sand porosity
poro = 0.4

# Sand density [kg/m3]
rohs      = 2650 

# Water density [kg/m3]
roh       = 1000 

# Slope hypothesis 
#slope = 0.05

# gravitationnal acceleration [m/s2]
g = 9.81

# active profil heigh --> in the future it will be DOC value +berme value [m] we can also use Dc as 

DoC = 10 

# Earth radius [m]
R = 6371000


In [ ]:
for i in range(1,len(Marnage)):
        if str(Marnage[i]) == 'nan':
            Marnage[i]=Marnage[i-1]
            
print("# - Marnage is ok")
           
for ligne in RiverD:
    for i in range(len(ligne)):
        # Vérifier si l'élément est "nan" et le remplacer par 0
        if str(ligne[i]) == 'nan':
            ligne[i] = 0

QrivD = FUNC.RIVDIS(RiverD,BQART)
for ligne in QrivD:
    if str(ligne) == 'nan':
        print(ligne)
        
print("# - QrivD is ok")

L_SU = (g/(2*np.pi))*Tp**2

beta       = 0.12 * ((np.sqrt(2*np.pi*d50*L_SU))/(Hs * (1+(Marnage/Hs))))**(1/2)

for i in range(len(beta)):
    for j in range(len(beta[0])):
        if str(beta[i,j]) == 'nan':
            print(i,j)
            
print("# - foreshore slopes is ok ")

# Set up des vagues [m] 
SU   = 0.35*beta*np.sqrt(Hs*L_SU)

print("# - Set up is ok")

TWL  = SLA + DAC+ SU

print("# - TWL is ok")


In [ ]:
# Calculer les percentiles globaux entre toutes les positions et tous les temps
global_percentiles = np.percentile(Marnage, [5, 95])

# Afficher les résultats
print("Percentiles globaux (5%, 95%):", global_percentiles)

fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(15, 10))  # Correction ici

sc = ax1.scatter(lon, lat, c=Marnage, s=10)
ax2.hist(Marnage.flatten(), bins=50, alpha=0.7, color='grey', label='Distribution')
ax2.axvline(global_percentiles[0], color='red', linestyle='dashed', linewidth=1, label='5th Percentile')
ax2.axvline(global_percentiles[1], color='green', linestyle='dashed', linewidth=1, label='95th Percentile')

ax2.set_xlabel('Tide')
ax2.set_ylabel('Frequency')
ax2.legend()
# Ajout d'une colorbar
cbar = plt.colorbar(sc, ax=ax1, label='Marnage max [m]')

ax1.set_xlabel('Longitude (°)')
ax1.set_ylabel('Latitude (°)')
ax1.set_title('Marnage max over the period 2000-2019')

plt.show()

In [ ]:
# Calculer les percentiles globaux entre toutes les positions et tous les temps
global_percentiles = np.percentile(RiverD, [5, 95])

# Afficher les résultats
print("Percentiles globaux (5%, 95%):", global_percentiles)

fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(15, 10))  # Correction ici

sc = ax1.scatter(lon, lat, c=(np.mean(RiverD,axis=0)), s=10)
ax2.hist(RiverD.flatten(), bins=50, alpha=0.7, color='grey', label='Distribution')
ax2.axvline(global_percentiles[0], color='red', linestyle='dashed', linewidth=1, label='5th Percentile')
ax2.axvline(global_percentiles[1], color='green', linestyle='dashed', linewidth=1, label='95th Percentile')

ax2.set_xlabel('River discharge')
ax2.set_ylabel('Frequency')
ax2.legend()
# Ajout d'une colorbar
cbar = plt.colorbar(sc, ax=ax1, label='River discharge [m3/month]')

ax1.set_xlabel('Longitude (°)')
ax1.set_ylabel('Latitude (°)')
ax1.set_title('River discharge from ISBA over the period 2000-2019')

plt.show()

In [ ]:
# Calculer les percentiles globaux entre toutes les positions et tous les temps
global_percentiles = np.percentile(BQART, [5, 95])

# Afficher les résultats
print("Percentiles globaux (5%, 95%):", global_percentiles)

fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(15, 10))  # Correction ici

sc = ax1.scatter(lon, lat, c=BQART, s=10)
ax2.hist(BQART.flatten(), bins=50, alpha=0.7, color='grey', label='Distribution')
ax2.axvline(global_percentiles[0], color='red', linestyle='dashed', linewidth=1, label='5th Percentile')
ax2.axvline(global_percentiles[1], color='green', linestyle='dashed', linewidth=1, label='95th Percentile')

ax2.set_xlabel('BQART')
ax2.set_ylabel('Frequency')
ax2.legend()
# Ajout d'une colorbar
cbar = plt.colorbar(sc, ax=ax1, label='BQART [m3/month]')

ax1.set_xlabel('Longitude (°)')
ax1.set_ylabel('Latitude (°)')
ax1.set_title('BQART over the period 2000-2019')

plt.show()

In [ ]:
global_percentiles = np.percentile(QrivD, [5, 95])

# Afficher les résultats
print("Percentiles globaux (5%, 95%):", global_percentiles)

fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(15, 10))  # Correction ici

sc = ax1.scatter(lon, lat, c=np.mean(QrivD, axis=0), s=10)
ax2.hist(QrivD.flatten(), bins=50, alpha=0.7, color='grey', label='Distribution')
ax2.axvline(global_percentiles[0], color='red', linestyle='dashed', linewidth=1, label='Percentile 5%')
ax2.axvline(global_percentiles[1], color='green', linestyle='dashed', linewidth=1, label='Percentile 95%')

ax2.set_xlabel('River "solid" discharge (BQART+ISBA) [m3/month]',fontsize=16)
ax2.set_ylabel('Fréquence',fontsize=16)
ax2.legend(fontsize=16)
ax2.tick_params(labelsize=13)
cbar = plt.colorbar(sc, ax=ax1, label='[m3/month]')

ax1.set_xlabel('Longitude (°)')
ax1.set_ylabel('Latitude (°)')
ax1.set_title('MeanRiver "solid" discharge over the period 2000-2019')

plt.show()


In [ ]:
global_percentiles = np.percentile(Hs, [5, 95])

print("Percentiles globaux (5%, 95%):", global_percentiles)

fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(15, 10))  # Correction ici

sc = ax1.scatter(lon, lat, c=np.mean(Hs, axis=0), s=10)
ax2.hist(Hs.flatten(), bins=50, alpha=0.7, color='grey', label='Distribution')
ax2.axvline(global_percentiles[0], color='red', linestyle='dashed', linewidth=1, label='Percentile 5%')
ax2.axvline(global_percentiles[1], color='green', linestyle='dashed', linewidth=1, label='Percentile 95%')
ax2.set_xlabel('Hs [m]', fontsize=16)
ax2.set_ylabel('Frequency',fontsize=16)
ax2.tick_params(labelsize=13)
ax2.legend(fontsize=16)
cbar = plt.colorbar(sc, ax=ax1, label='Wave height [m]')

ax1.set_xlabel('Longitude (°)')
ax1.set_ylabel('Latitude (°)')
ax1.set_title('Mean wave height over the period 2000-2019')

plt.show()

In [ ]:
global_percentiles = np.percentile(Tp, [5, 95])

print("Percentiles globaux (5%, 95%):", global_percentiles)

fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(15, 10))  # Correction ici

sc = ax1.scatter(lon, lat, c=np.mean(Tp, axis=0), s=10)
ax2.hist(Tp.flatten(), bins=50, alpha=0.7, color='grey', label='Distribution')
ax2.axvline(global_percentiles[0], color='red', linestyle='dashed', linewidth=1, label='Percentile 5%')
ax2.axvline(global_percentiles[1], color='green', linestyle='dashed', linewidth=1, label='Percentile 95%')
ax2.set_xlabel('Tp [s]', fontsize=16)
ax2.set_ylabel('Frequency',fontsize=16)
ax2.tick_params(labelsize=13)
ax2.legend(fontsize=16)
cbar = plt.colorbar(sc, ax=ax1, label='Peak period [s]')

ax1.set_xlabel('Longitude (°)')
ax1.set_ylabel('Latitude (°)')
ax1.set_title('Mean peak period over the period 2000-2019')

plt.show()

In [ ]:
global_percentiles = np.percentile(Dir, [5, 95])

print("Percentiles globaux (5%, 95%):", global_percentiles)

fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(15, 10))  # Correction ici

sc = ax1.scatter(lon, lat, c=np.mean(Dir, axis=0), s=10)
ax2.hist(Dir.flatten(), bins=50, alpha=0.7, color='grey', label='Distribution')
ax2.axvline(global_percentiles[0], color='red', linestyle='dashed', linewidth=1, label='Percentile 5%')
ax2.axvline(global_percentiles[1], color='green', linestyle='dashed', linewidth=1, label='Percentile 95%')
ax2.set_xlabel('Dir [°]', fontsize=16)
ax2.set_ylabel('Frequency',fontsize=16)
ax2.tick_params(labelsize=13)
ax2.legend(fontsize=16)
cbar = plt.colorbar(sc, ax=ax1, label='Wave direction [°]')

ax1.set_xlabel('Longitude (°)')
ax1.set_ylabel('Latitude (°)')
ax1.set_title('Mean wave direction over the period 2000-2019')

plt.show()

In [ ]:
# Calculer les percentiles globaux entre toutes les positions et tous les temps
global_percentiles = np.percentile(SU, [5, 95])

# Afficher les résultats
print("Percentiles globaux (5%, 95%):", global_percentiles)

fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(15, 10))  # Correction ici

sc = ax1.scatter(lon, lat, c=np.mean(SU, axis=0), s=10)
ax2.hist(SU.flatten(), bins=50, alpha=0.7, color='grey', label='Distribution')
ax2.axvline(global_percentiles[0], color='red', linestyle='dashed', linewidth=1, label='Percentile 5%')
ax2.axvline(global_percentiles[1], color='green', linestyle='dashed', linewidth=1, label='Percentile 95%')

# Correction ici
ax2.set_xlabel('Set-Up [m]',fontsize=16)
ax2.set_ylabel('Frequency',fontsize=16)
ax2.legend(fontsize=16)
ax2.tick_params(labelsize=13)

# Ajoutez une colorbar
cbar = plt.colorbar(sc, ax=ax1, label='Wave set-up with slope corrected by tide [m]')

# Assurez-vous d'utiliser le bon axe pour les étiquettes et le titre
ax1.set_xlabel('Longitude (°)')
ax1.set_ylabel('Latitude (°)')
ax1.set_title('Mean wave set-up over the period 2000-2019')


In [ ]:
# Calculer les percentiles globaux entre toutes les positions et tous les temps
global_percentiles = np.percentile(DAC, [5, 95])

# Afficher les résultats
print("Percentiles globaux (5%, 95%):", global_percentiles)

fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(15, 10))  # Correction ici

sc = ax1.scatter(lon, lat, c=np.mean(DAC, axis=0), s=10)
ax2.hist(DAC.flatten(), bins=50, alpha=0.7, color='grey', label='Distribution')
ax2.axvline(global_percentiles[0], color='red', linestyle='dashed', linewidth=1, label='Percentile 5%')
ax2.axvline(global_percentiles[1], color='green', linestyle='dashed', linewidth=1, label='Percentile 95%')

# Correction ici
ax2.set_xlabel('Run-up [m]',fontsize=16)
ax2.set_ylabel('Frequency',fontsize=16)
ax2.legend(fontsize=16)
ax2.tick_params(labelsize=13)


# Ajoutez une colorbar
cbar = plt.colorbar(sc, ax=ax1, label='DAC [m]')

# Assurez-vous d'utiliser le bon axe pour les étiquettes et le titre
ax1.set_xlabel('Longitude (°)')
ax1.set_ylabel('Latitude (°)')
ax1.set_title('Mean DAC over the period 2000-2019')

plt.show()


In [ ]:
# Calculer les percentiles globaux entre toutes les positions et tous les temps
global_percentiles = np.percentile(SLA, [5, 95])

# Afficher les résultats
print("Percentiles globaux (5%, 95%):", global_percentiles)

fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(15, 10))  # Correction ici

sc = ax1.scatter(lon, lat, c=np.mean(SLA, axis=0), s=10)
ax2.hist(SLA.flatten(), bins=50, alpha=0.7, color='grey', label='Distribution')
ax2.axvline(global_percentiles[0], color='red', linestyle='dashed', linewidth=1, label='Percentile 5%')
ax2.axvline(global_percentiles[1], color='green', linestyle='dashed', linewidth=1, label='Percentile 95%')

# Correction ici
ax2.set_xlabel('Sea level anomaly [m]',fontsize=16)
ax2.set_ylabel('Frequency',fontsize=16)
ax2.legend(fontsize=16)
ax2.tick_params(labelsize=13)
ax2.xaxis.set_major_locator(plt.MaxNLocator(5))




# Ajoutez une colorbar
cbar = plt.colorbar(sc, ax=ax1, label='mean SLA [m]')

# Assurez-vous d'utiliser le bon axe pour les étiquettes et le titre
ax1.set_xlabel('Longitude (°)')
ax1.set_ylabel('Latitude (°)')
ax1.set_title('Mean of SLA over the period 2000-2019')

plt.show()



In [ ]:
# Calculer les percentiles globaux entre toutes les positions et tous les temps
global_percentiles = np.percentile(np.tan(beta), [5, 95])

# Afficher les résultats
print("Percentiles globaux (5%, 95%):", global_percentiles)

fig, (ax1, ax2) = plt.subplots(2,1, figsize=(15, 10))  # Correction ici

sc = ax1.scatter(lon, lat, c=np.mean(np.tan(beta), axis=0), s=10)
ax2.hist(np.tan(beta).flatten(), bins=50, alpha=0.7, color='grey', label='Distribution')
ax2.axvline(global_percentiles[0], color='red', linestyle='dashed', linewidth=1, label='5th Percentile')
ax2.axvline(global_percentiles[1], color='green', linestyle='dashed', linewidth=1, label='95th Percentile')

# Correction ici
ax2.set_xlabel('Foreshore slope')
ax2.set_ylabel('Frequency')
ax2.legend()

# Ajoutez une colorbar
cbar = plt.colorbar(sc, ax=ax1, label='slope')

# Assurez-vous d'utiliser le bon axe pour les étiquettes et le titre
ax1.set_xlabel('Longitude (°)')
ax1.set_ylabel('Latitude (°)')
ax1.set_title('Mean slope over the period 2000-2019')

plt.show()


In [ ]:
# Calculer les percentiles globaux entre toutes les positions et tous les temps
global_percentiles = np.percentile(TWL, [5, 95])

# Afficher les résultats
print("Percentiles globaux (5%, 95%):", global_percentiles)

fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(15, 10))  # Correction ici

sc = ax1.scatter(lon, lat, c=np.mean(TWL, axis=0), s=10)
ax2.hist(TWL.flatten(), bins=50, alpha=0.7, color='grey', label='Distribution')
ax2.axvline(global_percentiles[0], color='red', linestyle='dashed', linewidth=1, label='Percentile 5%')
ax2.axvline(global_percentiles[1], color='green', linestyle='dashed', linewidth=1, label='Percentile 95%')

# Correction ici
ax2.set_xlabel("Total water level [m]",fontsize=16)
ax2.set_ylabel('Frequency',fontsize=16)
ax2.legend(fontsize=16)
ax2.tick_params(labelsize=13)
ax2.xaxis.set_major_locator(plt.MaxNLocator(5))

# Ajoutez une colorbar
cbar = plt.colorbar(sc, ax=ax1, label='Total water level with slope corrected[m]')

# Assurez-vous d'utiliser le bon axe pour les étiquettes et le titre
ax1.set_xlabel('Longitude (°)')
ax1.set_ylabel('Latitude (°)')
ax1.set_title('Mean total water level over the period 1993-2016')

plt.show()


# ZONE DELIMITATIONS

In [ ]:
index_deb, index_fin = FUNC.trouver_indice_section(lon, lat,0.55)
sections_fusionnees,indices_fusionnes = FUNC.fusionner_sections(index_deb, index_fin, lon, lat,0.55)
paires_filtrees = [(i, j) for i, j in indices_fusionnes if abs(i - j) > 2]

In [ ]:
# Création de la figure et de l'axe avec projection
fig, ax1 = plt.subplots(figsize=(14, 15), subplot_kw={'projection': ccrs.PlateCarree()})

# Ajout des caractéristiques de la carte
ax1.add_feature(cfeature.LAND, facecolor='gainsboro')
ax1.add_feature(cfeature.COASTLINE, linewidth=0.5, edgecolor='black')

# Configurer les limites de la carte
ax1.set_extent([-180, 180, -65, 65], crs=ccrs.PlateCarree())

# Configurer les étiquettes de longitude et latitude
ax1.set_xticks([-180, -120, -60, 0, 60, 120, 180], crs=ccrs.PlateCarree())
ax1.set_yticks([-60, -30, 0, 30, 60], crs=ccrs.PlateCarree())
ax1.xaxis.set_major_formatter(plt.FuncFormatter(lambda val, pos: f'{abs(val)}°{"W" if val < 0 else "E"}'))
ax1.yaxis.set_major_formatter(plt.FuncFormatter(lambda val, pos: f'{abs(val)}°{"S" if val < 0 else "N"}'))

# Taille de police pour les labels des axes
ax1.tick_params(axis='both', labelsize=16)

for i, section in enumerate(sections_fusionnees):
    plt.plot(section[:, 0], section[:, 1], label=f'Section {i + 1}')
plt.scatter(lon[3505:3543],lat[3505:3543])
plt.legend()
    # Affichage de la figure
plt.show()

CALIBRATION SUR TWL

In [ ]:
Xshores_VALIDATION = Xshores_val.transpose()

In [ ]:
lon_len = len(lon)
sigma = 1.0
Nmonth = 240

In [ ]:
date_list= pd.date_range('2000-1-1','2019-12-31', freq='M').strftime("%Y-%m-%d")
date = pd.DatetimeIndex(date_list)
num_dates = len(date_list)

In [ ]:
def gaussian_smooth(data, sigma):
    smoothed_data = np.apply_along_axis(lambda m: gaussian_filter1d(m, sigma=sigma), axis=0, arr=data)
    return smoothed_data    

def objective_function(c, TWL, beta, Xshores_VALIDATION, lon_index, sigma, date, Nmonth):
    X = np.zeros(Nmonth)
    for t in range(1, Nmonth):
        dx_hydro  = - (TWL[t, lon_index] - TWL[t - 1, lon_index]) / (c * np.tan(beta[t, lon_index]))
        dx_morpho = -((1 / (c * np.tan(beta[t, lon_index]))) - (1 / (c * np.tan(beta[t - 1, lon_index])))) 
        X[t] = X[t - 1] + (dx_hydro + dx_morpho)
    
    # Detrend des signaux
    X_detrend = np.apply_along_axis(detrend_linear,0,X)
    val_detrend = np.apply_along_axis(detrend_linear,0,Xshores_VALIDATION[:, lon_index])

    # Filtre gaussien
    X_detrend2 = gaussian_smooth(X_detrend, sigma)
    val_detrend2 = gaussian_smooth(val_detrend, sigma=sigma)

    # Calcul de la climatologie saisonnière
    climato_X_TOT = pd.DataFrame(X_detrend2, index=date)
    climato_val_TOT = pd.DataFrame(val_detrend2, index=date)
    
    dataframes_TOT = {
        'climato_X': climato_X_TOT,
        'climato_val': climato_val_TOT
    }
    
    climatologies_saisonnieres_TOT = calculer_climatologie_saisonniere(dataframes_TOT)
    climato_saisonniere_X_TOT = climatologies_saisonnieres_TOT['climato_X']
    climato_saisonniere_val_TOT = climatologies_saisonnieres_TOT['climato_val']
    
    # Calculer l'écart type
    std_obs = climato_saisonniere_val_TOT.std(axis=0)
    std_climato_X = climato_saisonniere_X_TOT.std(axis=0)
    
    # Comparaison des écarts types
    std_compar = abs(std_obs - std_climato_X)

    # Ajout d'un contrôle de corrélation
    correlation = climato_saisonniere_X_TOT.corrwith(climato_saisonniere_val_TOT).mean()
    if correlation < 0:
        std_compar += abs(correlation)  # Pénaliser les corrélations négatives

    return std_compar

# Fonction pour visualiser les résultats
def plot_results(results, date):
    fig, ax = plt.subplots(figsize=(12, 8))
    for (lon_index, c_results) in results.items():
        for c, (climato_X_TOT, climato_val_TOT) in c_results.items():
            ax.plot(climato_X_TOT.index, climato_X_TOT, label=f'Lon={lon_index}, c={c}')
        ax.plot(climato_val_TOT.index, climato_val_TOT, linestyle='--', label=f'Validation {lon_index}')
    
    # Ajouter les étiquettes et la légende
    ax.set_xlabel('Month', fontsize=16)
    ax.set_xticks(np.arange(1, 13, step=1))
    ax.set_xticklabels(['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec'])
    ax.tick_params(axis='both', labelsize=14)
    ax.legend(bbox_to_anchor=(1.05, 1), loc='upper left', fontsize=14)
    plt.show()

# Définir une gamme de valeurs pour c
c_values = [0.00000000000001, 0.0001,1]  # Ajustez selon vos besoins

# Dictionnaire pour stocker les résultats
results = {}
best_c_values = []
# Optimisation pour chaque position de longitude
for lon_index in range(lon_len):  # Assurez-vous que lon_index parcourt toutes les positions de longitude
    best_c = None
    best_result = None
    for c in c_values:
        # Effectuer la minimisation pour chaque valeur de c
        result = minimize(objective_function, x0=c, args=(TWL, beta, Xshores_VALIDATION, lon_index, sigma, date, Nmonth), bounds=[(0.00000000000001,1)])
        
        # Choisir la meilleure valeur de c en fonction de la fonction objectif
        if best_result is None or result.fun < best_result:
            best_result = result.fun
            best_c = result.x[0]
            
    best_c_values.append(best_c)
    
    # Afficher la meilleure valeur de c pour chaque longitude
#     print(f'Longitude Index: {lon_index}, Best c: {best_c}')

    # Calculer les résultats pour le meilleur c
    X = np.zeros(Nmonth)
    for t in range(1, Nmonth):
        dx_hydro  = - (TWL[t, lon_index] - TWL[t - 1, lon_index]) / (best_c * np.tan(beta[t, lon_index]))
        dx_morpho = -((1 / (best_c * np.tan(beta[t, lon_index]))) - (1 / (best_c * np.tan(beta[t - 1, lon_index])))) 
        X[t] = X[t - 1] + (dx_hydro + dx_morpho)
    
    X_detrend = detrend_linear(X)
    val_detrend = detrend_linear(Xshores_VALIDATION[:, lon_index])

    X_detrend2 = gaussian_filter1d(X_detrend, sigma=sigma)
    val_detrend2 = gaussian_filter1d(val_detrend, sigma=sigma)

    climato_X_TOT = pd.DataFrame(X_detrend2, index=date)
    climato_val_TOT = pd.DataFrame(val_detrend2, index=date)

    dataframes_TOT = {
        'climato_X': climato_X_TOT,
        'climato_val': climato_val_TOT
    }
    
    climatologies_saisonnieres_TOT = calculer_climatologie_saisonniere(dataframes_TOT)
    climato_saisonniere_X_TOT = climatologies_saisonnieres_TOT['climato_X']
    climato_saisonniere_val_TOT = climatologies_saisonnieres_TOT['climato_val']
    
    # Stocker les résultats avec la meilleure valeur de c
    results[lon_index] = {best_c: (climato_saisonniere_X_TOT, climato_saisonniere_val_TOT)}

# Visualiser les résultats
# plot_results(results, date)



In [ ]:
# np.save('best_c_values_141124.npy', best_c_values)
best_c_values = np.load('best_c_values_141124.npy')

In [ ]:
# Création de la figure et de l'axe avec projection
fig, ax1 = plt.subplots(figsize=(14, 15), subplot_kw={'projection': ccrs.PlateCarree()})

# Ajout des caractéristiques de la carte
ax1.add_feature(cfeature.LAND, facecolor='gainsboro')
ax1.add_feature(cfeature.COASTLINE, linewidth=0.5, edgecolor='black')

# Configurer les limites de la carte
ax1.set_extent([-180, 180, -65, 65], crs=ccrs.PlateCarree())

# Configurer les étiquettes de longitude et latitude
ax1.set_xticks([-180, -120, -60, 0, 60, 120, 180], crs=ccrs.PlateCarree())
ax1.set_yticks([-60, -30, 0, 30, 60], crs=ccrs.PlateCarree())
ax1.xaxis.set_major_formatter(plt.FuncFormatter(lambda val, pos: f'{abs(val)}°{"W" if val < 0 else "E"}'))
ax1.yaxis.set_major_formatter(plt.FuncFormatter(lambda val, pos: f'{abs(val)}°{"S" if val < 0 else "N"}'))

# Taille de police pour les labels des axes
ax1.tick_params(axis='both', labelsize=16)

# Scatter plot
sc1 = ax1.scatter(lon, lat, c=best_c_values, s=10,cmap='RdBu', transform=ccrs.PlateCarree())

# Ajout d'une colorbar
cbar = plt.colorbar(sc1, ax=ax1, orientation='vertical', fraction=0.017, pad=0.04, shrink=0.8)
cbar.set_label('c', fontsize=16)

# Ajustement de la taille des labels de la colorbar
cbar.ax.tick_params(labelsize=16)

# Affichage de la figure
plt.show()

In [ ]:
slop_c = best_c_values * np.tan(beta)
slope_c_med = np.median(slop_c,axis=0)

In [ ]:
global_percentiles = np.percentile(slope_c_med, [5, 95])

fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(15, 10))  # Correction ici

sc = ax1.scatter(lon, lat, c=slope_c_med, s=10)

# Ajoutez une colorbar
cbar = plt.colorbar(sc, ax=ax1, label='Median (c*tan(slope)')

# Assurez-vous d'utiliser le bon axe pour les étiquettes et le titre
ax1.set_xlabel('Longitude (°)')
ax1.set_ylabel('Latitude (°)')

ax2.hist(slope_c_med.flatten(), bins=50, alpha=0.7, color='grey', label='Distribution')
ax2.axvline(global_percentiles[0], color='red', linestyle='dashed', linewidth=1, label='Percentile 5%')
ax2.axvline(global_percentiles[1], color='green', linestyle='dashed', linewidth=1, label='Percentile 95%')

# Correction ici
ax2.set_xlabel("Median c*tan(beta)",fontsize=16)
ax2.set_ylabel('Frequency',fontsize=16)
ax2.legend(fontsize=16)
ax2.tick_params(labelsize=13)
ax2.xaxis.set_major_locator(plt.MaxNLocator(5))


plt.show()


In [ ]:
dt = 1 

In [ ]:
r_indices = []
r_dKAMP = []
r_dTWL = []
r_Ls = []
r_alpha = []
r_normal_finale = []
r_angle_inci_test = []

r_dx_CS_Hydro = []
r_dx_CS_MorphoLST = []
r_dx_CS_MorphoTOT = []
r_dx_CS_MorphoXshore = []
r_dx_CS_total = []

r_X_CS_LST = []
r_X_CS_River = []
r_X_CS_MorphoTOT = []
r_X_CS_Hydro = []
r_X_CS_Xshore = []
r_X_CS_total = []

r_Lon = []
r_Lat = []
r_Lon_m = []
r_Lat_m = []
r_DLon = []
r_DLat = []
r_dLon = []
r_dLat = []


In [ ]:
for i, section in tqdm(enumerate(paires_filtrees)):
    index_deb = section[0]
    index_fin = section[1]

    indices = np.arange(index_deb, index_fin)
    zone_len = len(indices)

    dKAMP = np.zeros((Nmonth, zone_len))
    dTWL = np.zeros((Nmonth, zone_len))
    Ls = np.zeros((Nmonth, zone_len))
    alpha = np.zeros((Nmonth, zone_len))
    normal_finale = np.zeros((Nmonth, zone_len))
    angle_inci_test = np.zeros((Nmonth, zone_len))

    dx_CS_Hydro = np.zeros((Nmonth, zone_len))
    dx_CS_MorphoLST = np.zeros((Nmonth, zone_len))
    dx_CS_MorphoTOT = np.zeros((Nmonth, zone_len))
    dx_CS_MorphoRIVER = np.zeros((Nmonth, zone_len))
    dx_CS_MorphoXshore = np.zeros((Nmonth, zone_len))
    dx_CS_total = np.zeros((Nmonth, zone_len))

    X_CS_LST = np.zeros((Nmonth, zone_len))
    X_CS_River = np.zeros((Nmonth, zone_len))
    X_CS_MorphoTOT = np.zeros((Nmonth, zone_len))
    X_CS_Hydro = np.zeros((Nmonth, zone_len))
    X_CS_Xshore = np.zeros((Nmonth, zone_len))
    X_CS_TOTAL = np.zeros((Nmonth, zone_len))

    Lon = np.zeros((Nmonth, zone_len), dtype=np.float64)
    Lat = np.zeros((Nmonth, zone_len), dtype=np.float64)
    Lon_m = np.zeros((Nmonth, zone_len), dtype=np.float64)
    Lat_m = np.zeros((Nmonth, zone_len), dtype=np.float64)
    DLon = np.zeros((Nmonth, zone_len))
    DLat = np.zeros((Nmonth, zone_len))
    dLon = np.zeros((Nmonth, zone_len))
    dLat = np.zeros((Nmonth, zone_len))

        # Calcul du point central géographique
    lon_mid = np.mean(lon[indices])
    lat_mid = np.mean(lat[indices])

        # INITIALISATION DU PREMIER PAS DE TEMPS
    Lon[0, :] = lon[indices]
    Lat[0, :] = lat[indices]

    alpha[0, :-1] = calculer_azimut(Lat[0, :], Lon[0, :])
    alpha[0, -1] = alpha[0, -2]

    normal_finale[0, :] = calculer_normale_orientee(alpha[0, :], Dir[0, indices])

    Lon_m[0, :], Lat_m[0, :] = lonlat2xy(Lon[0, :], Lat[0, :], lon_mid, lat_mid)

    dLon[0, :-1] = np.diff(Lon_m[0, :])
    dLat[0, :-1] = np.diff(Lat_m[0, :])
    dLon[0, -1] = 0
    dLat[0, -1] = 0

    Ls[0, :] = np.sqrt(dLon[0, :] ** 2 + dLat[0, :] ** 2)


        # Boucle principale de simulation
    for t in range(1, Nmonth):
        Lon[t, :] = Lon[t - 1, :]
        Lat[t, :] = Lat[t - 1, :]

        alpha[t, :-1] = calculer_azimut(Lat[t, :], Lon[t, :])
        alpha[t, -1] = alpha[t, -2]

        normal_finale[t, :] = calculer_normale_orientee(alpha[t, :], Dir[t, indices])

        Lon_m[t, :], Lat_m[t, :] = lonlat2xy(Lon[t, :], Lat[t, :], lon_mid, lat_mid)

        angle_inci_test[t, :] = (np.radians(Dir[t, indices]) - normal_finale[t, :])

        dLon[t, :-1] = np.diff(Lon_m[t, :])
        dLat[t, :-1] = np.diff(Lat_m[t, :])

        Ls[t, :-1] = np.sqrt(dLon[t, :-1] ** 2 + dLat[t, :-1] ** 2)

        smooth_Ls = 1.5 * Ls[t, :]  # m

        for j in range(zone_len - 1):
            idx = indices[j]
 
                # CALCUL COMPOSANTE HYDRO - LOCAL
            dTWL[t, j] = TWL[t, idx] - TWL[t - 1, idx]

            dx_CS_Hydro[t, j] = - (TWL[t, idx] - TWL[t - 1, idx]) / (best_c_values[idx] * np.tan(beta[t, idx]))  # m

                # Calcul morpho-local
            angle_loc_rad_ip1 = angle_inci_test[t, j + 1]
            angle_loc_rad_i = angle_inci_test[t, j]  # radians

            KAMP_mass_i = 2.33 * (rohs / (rohs - roh)) * (Tp[t, idx] ** 1.5) * (best_c_values[idx] * np.tan(beta[t, idx]) ** 0.75) * (d50 ** -0.25) * (Hs[t, idx] ** 2) * abs(np.sin(2 * angle_loc_rad_i)) ** 0.6 * np.sign(angle_loc_rad_i)
            KAMP_i = 86400 * 30 * (KAMP_mass_i / (rohs - roh)) / (1.0 - poro)  # m3/month

            KAMP_mass_ip1 = 2.33 * (rohs / (rohs - roh)) * (Tp[t, idx + 1] ** 1.5) * (best_c_values[idx] * np.tan(beta[t, idx + 1]) ** 0.75) * (d50 ** -0.25) * (Hs[t, idx + 1] ** 2) * abs(np.sin(2 * angle_loc_rad_ip1)) ** 0.6 * np.sign(angle_loc_rad_ip1)
            KAMP_ip1 = 86400 * 30 * (KAMP_mass_ip1 / (rohs - roh)) / (1.0 - poro)  # m3/month

            DKAMP = KAMP_ip1 - KAMP_i
            dKAMP[t, j] = DKAMP

            # LES delta calculé pour chaque composant
            dx_CS_MorphoTOT[t, j] = ((-1 / DoC) * ((DKAMP + QrivD[t, idx]) / Ls[t, j]) - ((1 / (best_c_values[idx]* np.tan(beta[t, idx]))) - (1 / (best_c_values[idx] * np.tan(beta[t - 1, idx]))))) * dt
            dx_CS_MorphoLST[t, j] = ((-1 / DoC) * ((DKAMP + QrivD[t, idx]) / Ls[t, j])) * dt
            dx_CS_MorphoXshore[t, j] = -((1 / (best_c_values[idx] * np.tan(beta[t,idx]))) - (1 / (best_c_values[idx] * np.tan(beta[t - 1, idx])))) * dt
            dx_CS_MorphoRIVER[t, j] = ((-1 / DoC) * (QrivD[t, idx] / Ls[t, j])) * dt
            
            dx_CS_total[t,j]        = dx_CS_Hydro[t,j] + dx_CS_MorphoTOT[t,j]

            # AJOUTS A LA POSITION DES SECTIONS

        X_CS_Hydro[t, :] = X_CS_Hydro[t - 1, :] + dx_CS_Hydro[t, :]
        X_CS_MorphoTOT[t, :] = X_CS_MorphoTOT[t - 1, :] + dx_CS_MorphoTOT[t, :]
        X_CS_LST[t, :] = X_CS_LST[t - 1, :] + dx_CS_MorphoLST[t, :]
        X_CS_Xshore[t, :] = X_CS_Xshore[t - 1, :] + dx_CS_MorphoXshore[t, :]
        X_CS_River[t, :] = X_CS_River[t - 1, :] + dx_CS_MorphoRIVER[t, :]

        X_CS_TOTAL[t,:]  = X_CS_TOTAL[t-1,:] + dx_CS_total[t,:]
        
        DLon[t,:] = - dx_CS_total[t,:]  * np.sin(normal_finale[t,:])
        DLat[t,:] =   dx_CS_total[t,:]  * np.cos(normal_finale[t,:])

        DLon[t,:] = FUNC.Wfilter(DLon[t,:],Lon_m[t,:],Lat_m[t,:],smooth_Ls)
        DLat[t,:] = FUNC.Wfilter(DLat[t,:],Lon_m[t,:],Lat_m[t,:],smooth_Ls)
        
        Lon_m[t,:] = Lon_m[t,:] + DLon[t,:]
        Lat_m[t,:] = Lat_m[t,:] + DLat[t,:]
              
        Lon[t,:],Lat[t,:] = xy2lonlat(Lon_m[t,:], Lat_m[t,:], lon_mid, lat_mid)

    r_indices.append(indices)
    r_dKAMP.append(dKAMP)
    r_dTWL.append(dTWL)
    r_Ls.append(Ls)
    r_alpha.append(alpha)
    r_normal_finale.append(normal_finale)
    r_angle_inci_test.append(angle_inci_test)
    
    r_dx_CS_Hydro.append(dx_CS_Hydro)
    r_dx_CS_MorphoLST.append(dx_CS_MorphoLST)
    r_dx_CS_MorphoTOT.append(dx_CS_MorphoTOT)
    r_dx_CS_MorphoXshore.append(dx_CS_MorphoXshore)
    r_dx_CS_total.append(dx_CS_total)
    
    r_X_CS_LST.append(X_CS_LST)
    r_X_CS_River.append(X_CS_River)
    r_X_CS_MorphoTOT.append(X_CS_MorphoTOT)
    r_X_CS_Hydro.append(X_CS_Hydro)
    r_X_CS_Xshore.append(X_CS_Xshore)
    r_X_CS_total.append(X_CS_TOTAL)

    r_Lon.append(Lon)
    r_Lat.append(Lat)
    r_Lon_m.append(Lon_m)
    r_Lat_m.append(Lat_m)
    r_DLon.append(DLon)
    r_DLat.append(DLat)
    r_dLon.append(dLon)
    r_dLat.append(dLat)

In [ ]:
# Convertir les listes de résultats en tableaux numpy 
results_dKAMP              = np.concatenate(r_dKAMP, axis=1)
results_dTWL               = np.concatenate(r_dTWL, axis=1)
results_Ls                 = np.concatenate(r_Ls, axis=1)
results_alpha              = np.concatenate(r_alpha, axis=1)

results_normal_finale      = np.concatenate(r_normal_finale, axis=1)
results_angle_inci_test    = np.concatenate(r_angle_inci_test, axis=1)

results_dx_CS_Hydro        = np.concatenate(r_dx_CS_Hydro, axis=1)
results_dx_CS_MorphoLST    = np.concatenate(r_dx_CS_MorphoLST, axis=1)
results_dx_CS_MorphoTOT    = np.concatenate(r_dx_CS_MorphoTOT, axis=1)
results_dx_CS_MorphoXshore = np.concatenate(r_dx_CS_MorphoXshore, axis=1)
results_dx_CS_total        = np.concatenate(r_dx_CS_total, axis=1)

results_X_CS_LST           = np.concatenate(r_X_CS_LST, axis=1)
results_X_CS_River         = np.concatenate(r_X_CS_River, axis=1)
results_X_CS_MorphoTOT     = np.concatenate(r_X_CS_MorphoTOT, axis=1)
results_X_CS_Hydro         = np.concatenate(r_X_CS_Hydro, axis=1)
results_X_CS_Xshore        = np.concatenate(r_X_CS_Xshore, axis=1)
results_X_CS_total         = np.concatenate(r_X_CS_total, axis=1)
  
results_Lon                = np.concatenate(r_Lon, axis=1)
results_Lat                = np.concatenate(r_Lat, axis=1)
results_Lon_m              = np.concatenate(r_Lat_m, axis=1)
results_DLon               = np.concatenate(r_DLon, axis=1)
results_DLat               = np.concatenate(r_DLat, axis=1)
results_dLon               = np.concatenate(r_dLon, axis=1)
results_dLat               = np.concatenate(r_dLat, axis=1)
results_indices            = np.concatenate(r_indices)

In [ ]:
## detrend mes signaux avec detrend
Total_detrend  = np.apply_along_axis(detrend_linear,0,results_X_CS_total)
Hydro_detrend  = np.apply_along_axis(detrend_linear,0,results_X_CS_Hydro)
Morpho_detrend = np.apply_along_axis(detrend_linear,0,results_X_CS_LST)
Xshores_detrend = np.apply_along_axis(detrend_linear,0,results_X_CS_Xshore)
val_detrend   = np.apply_along_axis(detrend_linear,0,Xshores_VALIDATION)

##smooth sur trois mois
from scipy.ndimage import gaussian_filter1d

# Define the sigma (standard deviation) for Gaussian smoothing
# Assuming an approximate 30-day window, adjust sigma based on your data frequency
sigma = 1.0  # This will smooth over approximately 3 months, adjust as needed

def gaussian_smooth(data, sigma):
    smoothed_data = np.apply_along_axis(lambda m: gaussian_filter1d(m, sigma=sigma), axis=0, arr=data)
    return smoothed_data

Total_detrend2 = gaussian_smooth(Total_detrend, sigma)
Hydro_detrend2 = gaussian_smooth(Hydro_detrend, sigma)
Morpho_detrend2 = gaussian_smooth(Morpho_detrend, sigma)
Xshores_detrend2 = gaussian_smooth(Xshores_detrend, sigma)

val_detrend2   = gaussian_smooth(val_detrend, sigma)

In [ ]:
validation_set = val_detrend2[:,results_indices]
lonX_set       = lonX[results_indices].flatten()
latX_set       = latX[results_indices].flatten()

In [ ]:
print(Total_detrend2.shape)

In [ ]:
from scipy.stats import pearsonr

correlations = []

for i in range(len(results_Lon[0])):
    # Extract data columns
    detrend_data = Total_detrend2[:, i]
    validation_data = validation_set[:, i]
    
    correlation = pearsonr(detrend_data, validation_data)

    # Append results to lists
    correlations.append(correlation)

In [ ]:
# Création de la figure et de l'axe avec projection
fig, ax1 = plt.subplots(figsize=(14, 15), subplot_kw={'projection': ccrs.PlateCarree()})

# Ajout des caractéristiques de la carte
ax1.add_feature(cfeature.LAND, facecolor='gainsboro')
ax1.add_feature(cfeature.COASTLINE, linewidth=0.5, edgecolor='black')

# Configurer les limites de la carte
ax1.set_extent([-180, 180, -65, 65], crs=ccrs.PlateCarree())

# Configurer les étiquettes de longitude et latitude
ax1.set_xticks([-180, -120, -60, 0, 60, 120, 180], crs=ccrs.PlateCarree())
ax1.set_yticks([-60, -30, 0, 30, 60], crs=ccrs.PlateCarree())
ax1.xaxis.set_major_formatter(plt.FuncFormatter(lambda val, pos: f'{abs(val)}°{"W" if val < 0 else "E"}'))
ax1.yaxis.set_major_formatter(plt.FuncFormatter(lambda val, pos: f'{abs(val)}°{"S" if val < 0 else "N"}'))

# Taille de police pour les labels des axes
ax1.tick_params(axis='both', labelsize=16)

# Scatter plot
sc1 = ax1.scatter(results_Lon[0], results_Lat[0], c=correlations, s=10, cmap='RdBu', transform=ccrs.PlateCarree())

# Ajout d'une colorbar
cbar = plt.colorbar(sc1, ax=ax1, orientation='vertical', fraction=0.017, pad=0.04, shrink=0.8)
cbar.set_label('Correlation', fontsize=16)

# Ajustement de la taille des labels de la colorbar
cbar.ax.tick_params(labelsize=16)

# Affichage de la figure
plt.show()

In [ ]:
# calcul relative deviation 
ecart = validation_set - Total_detrend2
ampli_obs = 4*np.std(validation_set,axis=0)

ecart_tot = (std(ecart,axis=0)/ampli_obs)*100

In [ ]:
# Création de la figure et de l'axe avec projection
fig, ax1 = plt.subplots(figsize=(14, 15), subplot_kw={'projection': ccrs.PlateCarree()})

# Ajout des caractéristiques de la carte
ax1.add_feature(cfeature.LAND, facecolor='gainsboro')
ax1.add_feature(cfeature.COASTLINE, linewidth=0.5, edgecolor='black')

# Configurer les limites de la carte
ax1.set_extent([-180, 180, -65, 65], crs=ccrs.PlateCarree())

# Configurer les étiquettes de longitude et latitude
ax1.set_xticks([-180, -120, -60, 0, 60, 120, 180], crs=ccrs.PlateCarree())
ax1.set_yticks([-60, -30, 0, 30, 60], crs=ccrs.PlateCarree())
ax1.xaxis.set_major_formatter(plt.FuncFormatter(lambda val, pos: f'{abs(val)}°{"W" if val < 0 else "E"}'))
ax1.yaxis.set_major_formatter(plt.FuncFormatter(lambda val, pos: f'{abs(val)}°{"S" if val < 0 else "N"}'))

# Taille de police pour les labels des axes
ax1.tick_params(axis='both', labelsize=16)

# Scatter plot
sc1 = ax1.scatter(results_Lon[0], results_Lat[0], c=ecart_tot, cmap='Blues',vmin=np.percentile(ecart_tot,5),vmax=np.percentile(ecart_tot,95), s=10,transform=ccrs.PlateCarree())

# Ajout d'une colorbar
cbar = plt.colorbar(sc1, ax=ax1, orientation='vertical', fraction=0.017, pad=0.04, shrink=0.8)
cbar.set_label('Relative deviation [%]', fontsize=16)

# Ajustement de la taille des labels de la colorbar
cbar.ax.tick_params(labelsize=16)

# Affichage de la figure
plt.show()

## DETERMINATION DES ZONES IPCC sur nos données 

In [ ]:
shapefile_path = "C:/Users/arias/Downloads/referenceRegions/referenceRegions.shp"
regions = gpd.read_file(shapefile_path)
# Créer un DataFrame avec les coordonnées des points
points_df = pd.DataFrame({
    'lon': lonX_set,  # Liste de longitudes
    'lat': latX_set
})

In [ ]:
import matplotlib.patches as patches
from shapely.geometry import Point

In [ ]:

# Créer un GeoDataFrame pour les points avec un index supplémentaire
points_df['index'] = points_df.index
geometry = [Point(xy) for xy in zip(points_df.lon, points_df.lat)]
points_gdf = gpd.GeoDataFrame(points_df, geometry=geometry, crs='EPSG:4326')

# Reprojeter les régions en WGS84 si nécessaire
if regions.crs is not None and regions.crs.to_string() != 'EPSG:4326':
    regions = regions.to_crs(epsg=4326)

# Définir une fonction pour vérifier si un polygone contient des points
def polygon_contains_points(polygon, points_gdf):
    return points_gdf[points_gdf.geometry.within(polygon)].shape[0] > 0

# Appliquer la fonction pour filtrer les régions
filtered_regions = regions[regions.geometry.apply(lambda x: polygon_contains_points(x, points_gdf))]

# Optionnel : Affiner le filtrage basé sur les limites de latitude
regions_filtered = filtered_regions[(filtered_regions.bounds.miny >= -70) & (filtered_regions.bounds.maxy <= 70)]

# Créer une colonne 'polygon_id' pour identifier chaque polygone
regions_filtered = regions_filtered.copy()  # Créer une copie pour éviter les warnings
regions_filtered['polygon_id'] = regions_filtered.index

# Définir le nombre de couleurs nécessaires
num_polygons = len(regions_filtered)
cmap = plt.get_cmap('tab20')  # Vous pouvez essayer 'tab20b', 'tab20c', 'viridis', etc.
colors = [cmap(i / num_polygons) for i in range(num_polygons)]

regions_filtered['color'] = [colors[i % len(colors)] for i in range(num_polygons)]

# Effectuer la jointure spatiale entre les points et les polygones filtrés
points_with_zones = gpd.sjoin(points_gdf, regions_filtered, how='left', predicate='within')

# Filtrer les points qui n'ont pas été assignés à un polygone
points_with_zones = points_with_zones.dropna(subset=['polygon_id'])

# Assigner la couleur des points en fonction du polygone auquel ils appartiennent
points_with_zones['color'] = points_with_zones['polygon_id'].map(regions_filtered.set_index('polygon_id')['color'])

# Préparer les données pour la légende
legend_data = regions_filtered[['polygon_id', 'color', 'NAME']].drop_duplicates().sort_values(by='polygon_id')

# Ajouter la colonne des indices correspondant dans points_df
points_with_zones['original_index'] = points_with_zones['index']

# Tracer les polygones filtrés
fig, ax = plt.subplots(figsize=(14, 50), subplot_kw={'projection': ccrs.PlateCarree()})

# Ajouter les caractéristiques de la carte
ax.add_feature(cfeature.LAND, facecolor='gainsboro')
ax.add_feature(cfeature.COASTLINE, linewidth=0.5, edgecolor='black')

# Tracer les polygones filtrés avec les couleurs de contour
for _, row in regions_filtered.iterrows():
    ax.add_patch(plt.Polygon(list(row['geometry'].exterior.coords), closed=True, edgecolor='black', facecolor='none', linewidth=2))

# Tracer les points avec les couleurs associées
points_with_zones.plot(ax=ax, color=points_with_zones['color'], markersize=10, alpha=0.75)

# # Ajouter la légende avec la taille de police ajustée
# patches_list = [patches.Patch(color=row['color'], label=row['NAME']) for _, row in legend_data.iterrows()]
# ax.legend(handles=patches_list, bbox_to_anchor=(1.05, 1), loc='upper left', fontsize=16)

# Configurer les limites de la carte
ax.set_extent([-180, 180, -65, 65], crs=ccrs.PlateCarree())

# Configurer les étiquettes de longitude et latitude
ax.set_xticks([-180, -120, -60, 0, 60, 120, 180], crs=ccrs.PlateCarree())
ax.set_yticks([-60, -30, 0, 30, 60], crs=ccrs.PlateCarree())
ax.xaxis.set_major_formatter(plt.FuncFormatter(lambda val, pos: f'{abs(val)}°{"W" if val < 0 else "E"}'))
ax.yaxis.set_major_formatter(plt.FuncFormatter(lambda val, pos: f'{abs(val)}°{"S" if val < 0 else "N"}'))

# Taille de police pour les labels des axes
ax.tick_params(axis='both', labelsize=16)

# Afficher la carte
plt.show()

In [ ]:
liste_index = points_with_zones['index'].tolist()
liste_polygone = points_with_zones['polygon_id'].tolist()

In [ ]:
#Je selectionne les outputs
Xshore_IPCC =Xshores_detrend2[:,liste_index]
Morpho_IPCC = Morpho_detrend2[:,liste_index]
Hydro_IPCC  = Hydro_detrend2[:,liste_index]

In [ ]:
# Identifier les zones uniques
unique_zones = np.unique(liste_polygone)

# Initialiser un tableau pour les moyennes par zone
Xshore_IPCC_means = np.zeros((Xshore_IPCC.shape[0], len(unique_zones)))
Hydro_IPCC_means = np.zeros((Hydro_IPCC.shape[0], len(unique_zones)))
Morpho_IPCC_means = np.zeros((Morpho_IPCC.shape[0], len(unique_zones)))

# Calculer la moyenne pour chaque zone
for i, zone in enumerate(unique_zones):
    # Trouver les indices des positions correspondant à cette zone
    indices = np.where(liste_polygone == zone)[0]
    # Calculer la moyenne des valeurs de X pour cette zone
    Xshore_IPCC_means[:, i] = np.mean(Xshore_IPCC[:, indices], axis=1)
    Hydro_IPCC_means[:, i] = np.mean(Hydro_IPCC[:, indices], axis=1)
    Morpho_IPCC_means[:, i] = np.mean(abs(Morpho_IPCC[:, indices]), axis=1)

print(Xshore_IPCC_means.shape)
print(Hydro_IPCC_means.shape)
print(Morpho_IPCC_means.shape)


In [ ]:
print(Xshore_IPCC.shape)

In [ ]:
Xshores_IPCC_std = np.nanstd(Xshore_IPCC,axis=0)
Morpho_IPCC_std = np.nanstd(Morpho_IPCC,axis=0)
Hydro_IPCC_std = np.nanstd(Hydro_IPCC,axis=0)

print(Xshores_IPCC_std.shape)
print(Morpho_IPCC_std)
print(Hydro_IPCC_std)

In [ ]:
HXM = Xshores_IPCC_std + Morpho_IPCC_std + Hydro_IPCC_std
print(HXM)

In [ ]:
contributions_Hydro = (Hydro_IPCC_std/HXM)*100
contributions_Morpho = (Morpho_IPCC_std/HXM)*100
contributions_Xshore = (Xshores_IPCC_std/HXM)*100

In [ ]:
# contributions_Hydro = []
# contributions_Morpho = []
# contributions_Xshore = []

# for i in range(len(Xshores_IPCC_std)):
#     c_Hydro  = (Hydro_IPCC_std[i] / HXM[i]) * 100
#     c_Morpho = (Morpho_IPCC_std[i] / HXM[i]) * 100
#     c_Xshore = (Xshores_IPCC_std[i] / HXM[i]) * 100
    
#     contributions_Hydro.append(c_Hydro)
#     contributions_Morpho.append(abs(c_Morpho))
#     contributions_Xshore.append(c_Xshore)

In [ ]:
print(contributions_Hydro)

In [ ]:
contributions_df = pd.DataFrame({
#     'polygone_id': unique_zones,
    'liste_polyg' : liste_polygone,
    'Cross-shore': contributions_Xshore,
    'Hydrological': contributions_Hydro,
    'Alongshore': abs(contributions_Morpho)
}).set_index('liste_polyg')

mean_contributions_df = contributions_df.groupby('liste_polyg').mean()


In [ ]:
# Fonction pour tracer les barres
def plot_bar_plot(ax, lon, lat, contributions, color_map, **kwargs):
    bar_width = 4
    spacing = bar_width * 1.5

    for i, (contribution_type, value) in enumerate(contributions.items()):
        color = color_map.get(contribution_type, 'grey')
        if isinstance(value, (int, float)):
            # Position des barres, légèrement décalée pour éviter les chevauchements
            x_pos = lon + (i - 1) * spacing
            y_pos = lat
            # Tracer chaque barre
            rect = patches.Rectangle(
                (x_pos, y_pos),  # Position
                bar_width,  # Largeur
                value / 5,  # Hauteur
                linewidth=1,  # Largeur du contour
                edgecolor='black',  # Couleur du contour
                facecolor=color,  # Couleur de la barre
                **kwargs
            )
            ax.add_patch(rect)


In [ ]:
import matplotlib.patches as mpatches

def plot_reference_bar(ax):
    # Position de la barre (bas gauche)
    x_pos = -170
    y_pos = -65
    bar_width = 4
    bar_height = 20  # Représente le 100%

    # Tracer la barre
    rect = patches.Rectangle(
        (x_pos, y_pos),  # Position
        bar_width,  # Largeur
        bar_height,  # Hauteur
        linewidth=1,  # Largeur du contour
        edgecolor='black',  # Couleur du contour
        facecolor='grey',  # Couleur de la barre
        alpha=0.6  # Transparence
    )
    ax.add_patch(rect)
    ax.text(x_pos + bar_width/2, y_pos + bar_height + 2, '100%', ha='center', fontsize=16)

# Ajouter une légende pour les barres
def add_legend(ax, color_map):
    x_legend=0.25
    y_legend=0.25
    legend_patches = [mpatches.Patch(color=color, label=label) for label, color in color_map.items()]
    ax.legend(handles=legend_patches, bbox_to_anchor=(x_legend,y_legend), fontsize=16)

# Créer la carte
fig, ax = plt.subplots(figsize=(14, 50), subplot_kw={'projection': ccrs.PlateCarree()})

# Ajouter les caractéristiques de la carte
ax.add_feature(cfeature.LAND, facecolor='gainsboro')
ax.add_feature(cfeature.COASTLINE, linewidth=0.5, edgecolor='black', alpha=0.9)

# Tracer les points avec les couleurs associées
points_with_zones.plot(ax=ax, color=points_with_zones['color'], markersize=10, alpha=1)

# Tracer les polygones filtrés avec les couleurs de contour
for _, row in regions_filtered.iterrows():
    ax.add_patch(plt.Polygon(list(row['geometry'].exterior.coords), closed=True, edgecolor=row['color'], facecolor='none', linewidth=2))

# Tracer les barres pour chaque zone
for _, row in regions_filtered.iterrows():
    lon = row.geometry.centroid.x - 3
    lat = row.geometry.centroid.y
    # Accéder aux contributions en utilisant l'index
    contributions = mean_contributions_df.loc[row.name]
    color_map = {'Cross-shore': 'orange', 'Hydrological': 'blue', 'Alongshore': 'green'}
    plot_bar_plot(ax, lon, lat, contributions, color_map, zorder=1001)

# Tracer la barre de référence
plot_reference_bar(ax)

# Ajouter la légende
add_legend(ax, color_map)

# Configurer les limites de la carte
ax.set_extent([-180, 180, -70, 75], crs=ccrs.PlateCarree())

# Configurer les étiquettes de longitude et latitude
ax.set_xticks([-180, -120, -60, 0, 60, 120, 180], crs=ccrs.PlateCarree())
ax.set_yticks([-60, -30, 0, 30, 60], crs=ccrs.PlateCarree())
ax.xaxis.set_major_formatter(plt.FuncFormatter(lambda val, pos: f'{abs(val)}°{"W" if val < 0 else "E"}'))
ax.yaxis.set_major_formatter(plt.FuncFormatter(lambda val, pos: f'{abs(val)}°{"S" if val < 0 else "N"}'))

# Taille de police pour les labels des axes
ax.tick_params(axis='both', labelsize=16)

# Afficher la carte
plt.show()